<a href="https://colab.research.google.com/github/RegenMedandAI/Machine-Learning-and-cancer-studies/blob/main/AI_ML_developer_Capstone_Cancer_machinelearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

i am doing a machine learning study on breast cancer data from Kaggle, https://www.kaggle.com/datasets/brunogrisci/breast-cancer-gene-expression-cumida, the csv was edited in r to include Gene_symbol (gene name alongside the microarray probe ID) and the tumour grade. This generated the file Breast_GSE45827p.csv, from which the study proceeds. The microarray transcription data for breast cancer has sample cataagories of "normal", of tumor grade 0; "Luminal_A", of tumor grade 1; "HER", of tumor grade 3; "basal", of tumor grade 3; and "Luminal B", of luminal grade 2. The samples are in columns starting at column3, column name is sample number, the first row are the catagories and the second row are the grades. The first column are the gene list as probeID and the second column are the gene list as Gene_Symbol. The EDA consists of NAn check in the expression data, PCA analysis of the sample groups, a heatmap of the control genes to see if normalisation is needed

In [ ]:
#data files available https://drive.google.com/drive/folders/1b2eMbXCYnf6sMYoJ8Eak4OjDGQmRXw-j?usp=sharing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#load the data
data = pd.read_csv("Breast_GSE45827p.csv")

sample_ids = data.iloc[0, 2:153].index
categories = data.iloc[0, 2:153]
grades = data.iloc[1, 2:153]
sample_info = pd.DataFrame({
    'Sample_ID': sample_ids,
    'Category': categories,
    'Grade': grades
})
# Save the sample information to a CSV file
sample_info.to_csv('sample_info.csv', index=False)

# Extract gene information
gene_info = data.iloc[2:, :2]
gene_info.columns = ['ProbeID', 'GeneSymbol']
gene_info.reset_index(drop=True, inplace=True)

# Extract gene expression data
expression_data = data.iloc[2:, 2:153]
expression_data.columns = sample_ids
expression_data.set_index(gene_info['ProbeID'], inplace=True)

# Extract gene expression data
expression_data = data.iloc[2:, 2:153]
expression_data.columns = sample_ids

# Check for and handle any remaining NaN values
print("NaN values in expression data:", expression_data.isna().sum().sum())

# Transpose the data so that samples are rows and genes are columns
expression_data_transposed = expression_data.T
# Perform PCA
scaler = StandardScaler()
scaled_expression = scaler.fit_transform(expression_data_transposed)
pca = PCA(n_components=2)
pca_result = pca.fit_transform(scaled_expression)
# Print PCA results shape
print("PCA result shape:", pca_result.shape)

# # Create DataFrame for PCA results
pca_df = pd.DataFrame(data=pca_result, columns=['PC1', 'PC2'], index=sample_ids)
pca_df = pca_df.join(sample_info.set_index('Sample_ID'))
# Print first few rows of PCA DataFrame
print("\nFirst few rows of PCA DataFrame:")
print(pca_df.head())

import matplotlib.pyplot as plt
import seaborn as sns

# Create a figure with two subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Plot histogram of PC1
sns.histplot(pca_df['PC1'], kde=True, ax=ax1)
ax1.set_title('Distribution of PC1')
ax1.set_xlabel('PC1 Values')

# Plot histogram of PC2
sns.histplot(pca_df['PC2'], kde=True, ax=ax2)
ax2.set_title('Distribution of PC2')
ax2.set_xlabel('PC2 Values')

plt.tight_layout()
plt.show()

import matplotlib.pyplot as plt
import seaborn as sns

# Create a figure with two subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Plot histogram of PC1
sns.histplot(pca_df['PC1'], kde=True, ax=ax1)
ax1.set_title('Distribution of PC1')
ax1.set_xlabel('PC1 Values')

# Plot histogram of PC2
sns.histplot(pca_df['PC2'], kde=True, ax=ax2)
ax2.set_title('Distribution of PC2')
ax2.set_xlabel('PC2 Values')

plt.tight_layout()
plt.show()

In [ ]:
#PCA analysis showing tumour and grade

import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# Load the CSV file into a DataFrame
Breast_GSE45827p = pd.read_csv('Breast_GSE45827p.csv')

# 1. Skip row numbering (data already has rownames)
# 2. Extract relevant parts of the data
sample_ids6 = Breast_GSE45827p.iloc[0, 2:].values  # Sample IDs in row 0 (excluding name)
sample_grades6 = Breast_GSE45827p.iloc[1, 2:].values  # Sample types in row 1 (excluding name)
probe_ids6 = Breast_GSE45827p.iloc[3:, 0].values         # Probe IDs starting from row 4 (index)
gene_symbols6 = Breast_GSE45827p.iloc[3:, 1].values      # Gene symbols starting from row 4
gene_expression6 = Breast_GSE45827p.iloc[3:, 2:].T    # Transpose to have samples as rows

# 3. Print the first few sample types and grades
print("First few sample types:")
print(sample_grades6[:5])


# 4. Set row and column names for the gene expression matrix
gene_expression6.columns = probe_ids6
gene_expression6.index = sample_ids6

# 5. Convert gene expression data to numeric
gene_expression6 = gene_expression6.apply(pd.to_numeric)

# 6. Perform PCA
pca = PCA()
pca_result6 = pca.fit_transform(gene_expression6)

# 7. Extract PC1 and PC2
pc_data6 = pd.DataFrame({
    'PC1': pca_result6[:, 0],
    'PC2': pca_result6[:, 1],
    'Grade': sample_grades6,

})

# 8. Print the structure of pc_data6
print("Structure of pc_data6:")
print(pc_data6.info())

# 9. Calculate variance explained
var_explained6 = pca.explained_variance_ratio_

# 10. Plot PCA results colored by Grade
plt.figure(figsize=(10, 6))
sns.scatterplot(x='PC1', y='PC2', hue='Grade', data=pc_data6, s=100, alpha=0.7)
plt.title(f'PCA of Breast Cancer Gene Expression Data (Breast_GSE45827p) by Grade')
plt.xlabel(f'PC1 ({var_explained6[0] * 100:.2f}% variance explained)')
plt.ylabel(f'PC2 ({var_explained6[1] * 100:.2f}% variance explained)')
plt.legend(title='Cancer Grade')
plt.grid(True)
plt.show()


# 12. Print summary of PCA
print("PCA Summary:")
print(f"Variance explained by first two components: PC1 = {var_explained6[0] * 100:.2f}%, PC2 = {var_explained6[1] * 100:.2f}%")

# 13. Display the first few rows of the PC data
print("First few rows of PC data:")
print(pc_data6.head())

import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# Load the CSV file into a DataFrame
Breast_GSE45827p = pd.read_csv('Breast_GSE45827p.csv')

# 1. Skip row numbering (data already has rownames)
# 2. Extract relevant parts of the data
sample_ids6 = Breast_GSE45827p.iloc[0, 2:].values  # Sample IDs in row 0 (excluding name)
sample_types6 = Breast_GSE45827p.iloc[1, 2:].values  # Sample types in row 1 (excluding name)
probe_ids6 = Breast_GSE45827p.iloc[3:, 0].values         # Probe IDs starting from row 3 (index)
gene_symbols6 = Breast_GSE45827p.iloc[3:, 1].values      # Gene symbols starting from row 3
gene_expression6 = Breast_GSE45827p.iloc[3:, 2:].T    # Transpose to have samples as rows

# 3. Print the first few sample types
print("First few sample types (Cancer types):")
print(sample_ids6[:5])


# 4. Set row and column names for the gene expression matrix
gene_expression6.columns = probe_ids6
gene_expression6.index = sample_ids6

# 5. Convert gene expression data to numeric
gene_expression6 = gene_expression6.apply(pd.to_numeric)

# 6. Perform PCA
pca = PCA()
pca_result6 = pca.fit_transform(gene_expression6)

# 7. Extract PC1 and PC2
pc_data6 = pd.DataFrame({
    'PC1': pca_result6[:, 0],
    'PC2': pca_result6[:, 1],
    'Type': sample_ids6,
    })

# 8. Print the structure of pc_data6
print("Structure of pc_data6:")
print(pc_data6.info())

# 9. Calculate variance explained
var_explained6 = pca.explained_variance_ratio_

# 10. Plot PCA results colored by Type
plt.figure(figsize=(10, 6))
sns.scatterplot(x='PC1', y='PC2', hue='Type', data=pc_data6, s=100, alpha=0.7)
plt.title(f'PCA of Breast Cancer Gene Expression Data (Breast_GSE45827p) by Type')
plt.xlabel(f'PC1 ({var_explained6[0] * 100:.2f}% variance explained)')
plt.ylabel(f'PC2 ({var_explained6[1] * 100:.2f}% variance explained)')
plt.legend(title='Cancer Type')
plt.grid(True)
plt.show()


# 12. Print summary of PCA
print("PCA Summary:")
print(f"Variance explained by first two components: PC1 = {var_explained6[0] * 100:.2f}%, PC2 = {var_explained6[1] * 100:.2f}%")

# 13. Display the first few rows of the PC data
print("First few rows of PC data:")
print(pc_data6.head())

In [ ]:
# Hierarchical Clustering with Heatmap of AFFX and controls, test whether data normalisation is required

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data
df = pd.read_csv('Breast_GSE45827p.csv')

# Extract sample data (columns 3 to 153, rows 3 to 54677) and convert to float
sample_data = df.iloc[2:54677, 2:153].astype(float)

# Extract probe IDs and gene symbols
probe_ids = df.iloc[2:54677, 0]
gene_symbols = df.iloc[2:54677, 1]

# Extract sample names, types, and grades
sample_names = df.columns[2:153]
sample_types = df.iloc[0, 2:153]
sample_grades = df.iloc[1, 2:153]

# Filter for AFFX probes
affx_mask = probe_ids.str.contains('AFFX', case=False, na=False)
affx_data = sample_data[affx_mask]
affx_probe_ids = probe_ids[affx_mask]

# Create a DataFrame with AFFX probe data
affx_df = pd.DataFrame(affx_data.values, index=affx_probe_ids, columns=sample_names)

# Create the heatmap
plt.figure(figsize=(20, 10))
sns.heatmap(affx_df, cmap='coolwarm', center=0, cbar_kws={'label': 'Expression Value'})
plt.title('Heatmap of AFFX Probes Across Samples')
plt.xlabel('Samples')
plt.ylabel('AFFX Probe IDs')
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Print some statistics
print(f"Total number of AFFX probes: {len(affx_probe_ids)}")
print("\nFirst 10 AFFX probe IDs:")
print(affx_probe_ids[:10].tolist())

# Calculate and print average expression for each AFFX probe
avg_expression = affx_df.mean(axis=1).sort_values(ascending=False)
print("\nTop 10 AFFX probes by average expression:")
print(avg_expression.head(10))

print("\nBottom 10 AFFX probes by average expression:")
print(avg_expression.tail(10))

PART2 Breast Cancer Gene Expression Analysis: Multi-Model Approach Overview This project implements a comprehensive machine learning analysis for breast cancer classification using gene expression data, employing three distinct modeling approaches to assess their ability to best identify cancer type and tumour grade.

Random Forest (RF) Support Vector Machine (SVM) Deep Neural Network (DNN)

Data Preprocessing

Feature extraction from gene expression values Label encoding for cancer types and tumor grades Train-test split (80-20) StandardScaler application for feature normalization

Model Architectures

Random Forest Classifier
Non-parametric, ensemble learning method Uses 100 decision trees (n_estimators=100) Advantages:

Built-in feature importance Handles non-linearity effectively Less prone to overfitting

Support Vector Machine
Kernel: Linear and RBF (Radial Basis Function) Hyperparameters:

probability=True for probability estimates random_state=42 for reproducibility

Advantages:

Effective in high-dimensional spaces Memory efficient Versatile through different kernel functions

Deep Neural Network
Architecture:

Input layer: Matches feature dimensionality Hidden layers: 256 → 128 → 64 neurons Output layer: Softmax activation

Training parameters:

Epochs: 100 Batch size: 32 Early stopping with patience=10

Regularization techniques:

Dropout (0.3) BatchNormalization

Optimizer: Adam Loss function: Categorical crossentropy

Model Comparison Similarities

RF and SVM:

Both are traditional machine learning algorithms Work well with high-dimensional data Less computational resources compared to DNN

SVM and DNN:

Both create hyperplanes for classification Can model non-linear relationships

Key Differences

Random Forest:

Ensemble method using multiple decision trees Provides feature importance out-of-the-box May struggle with very high-dimensional data

SVM:

Creates optimal hyperplane for class separation Kernel trick for non-linear classification Can be computationally intensive for large datasets

Deep Neural Network:

Most complex model with multiple layers Requires more data for optimal performance Computationally intensive training Can automatically learn feature representations

Comparative Efficacy The most appropriate model depends on specific requirements:

For interpretability: Random Forest For balanced performance: SVM For complex pattern recognition: Deep Neural Network

Implementation Notes

Both cancer type and tumor grade classifications are performed All models use consistent preprocessing for fair comparison Early stopping in DNN prevents overfitting Multiple evaluation metrics: accuracy, classification report, confusion matrix

In [ ]:
#This code is designed for preprocessing breast cancer gene expression data in preparation for machine learning classification tasks. It involves loading the data, extracting features (gene expression values) and labels (cancer types and tumor grades), encoding categorical labels into numerical form, and splitting the dataset into training and testing sets. Additionally, feature scaling is performed to standardize the data, which helps improve model performance during classification of cancer types and tumor grades. The dataset used for this task is sourced from the Breast_GSE45827p.csv file, which contains gene expression profiles and associated clinical information.

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

# Load the data
df = pd.read_csv('Breast_GSE45827p.csv')
print(df.head()) # Use head() to display the first few rows of the dataframe

#Corrected Data Preprocessing for Breast Cancer Gene Expression Analysis
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

# Load the data


# Extract features (gene expression values)
X = df.iloc[2:, 2:153].T  # Transpose to get samples as rows
X.index = df.columns[2:153]  # Set sample names as index
X.columns = df.iloc[2:, 0]  # Set probe_id as column names

# Extract labels
y_type = df.iloc[0, 2:153]
y_grade = df.iloc[1, 2:153].astype(str)

# Create a mapping of probe_id to gene symbol
gene_map = dict(zip(df.iloc[2:, 0], df.iloc[2:, 1]))

# Encode cancer types and grades
le_type = LabelEncoder()
le_grade = LabelEncoder()
y_type_encoded = le_type.fit_transform(y_type)
y_grade_encoded = le_grade.fit_transform(y_grade)

# Split the data
X_train, X_test, y_type_train, y_type_test, y_grade_train, y_grade_test = train_test_split(
    X, y_type_encoded, y_grade_encoded, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Data preprocessed and split into training and test sets.")
print(f"Training set shape: {X_train_scaled.shape}")
print(f"Test set shape: {X_test_scaled.shape}")
print(f"Cancer types: {le_type.classes_}")
print(f"Tumor grades: {le_grade.classes_}")

In [ ]:
# Machine Learning Analysis for Breast Cancer Gene Expression Data- RANDOM CLASSIFIER
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Using the preprocessed data from previous code
# Assuming we have: X_train_scaled, X_test_scaled, y_type_train, y_type_test, y_grade_train, y_grade_test

# 1. Train and evaluate cancer type classifier
rf_type = RandomForestClassifier(n_estimators=100, random_state=42)
rf_type.fit(X_train_scaled, y_type_train)

# Predict and evaluate
y_type_pred = rf_type.predict(X_test_scaled)

print("Cancer Type Classification Results:")
print("Accuracy:", accuracy_score(y_type_test, y_type_pred))
print("\nClassification Report:")
print(classification_report(y_type_test, y_type_pred, target_names=le_type.classes_))

# Confusion Matrix for cancer types
plt.figure(figsize=(10, 8))
cm_type = confusion_matrix(y_type_test, y_type_pred)
sns.heatmap(cm_type, annot=True, fmt='d', cmap='Blues',
            xticklabels=le_type.classes_,
            yticklabels=le_type.classes_)
plt.title('Confusion Matrix - Cancer Types')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# 2. Train and evaluate tumor grade classifier
rf_grade = RandomForestClassifier(n_estimators=100, random_state=42)
rf_grade.fit(X_train_scaled, y_grade_train)

# Predict and evaluate
y_grade_pred = rf_grade.predict(X_test_scaled)

print("\nTumor Grade Classification Results:")
print("Accuracy:", accuracy_score(y_grade_test, y_grade_pred))
print("\nClassification Report:")
print(classification_report(y_grade_test, y_grade_pred, target_names=le_grade.classes_))

# Confusion Matrix for tumor grades
plt.figure(figsize=(10, 8))
cm_grade = confusion_matrix(y_grade_test, y_grade_pred)
sns.heatmap(cm_grade, annot=True, fmt='d', cmap='Blues',
            xticklabels=le_grade.classes_,
            yticklabels=le_grade.classes_)
plt.title('Confusion Matrix - Tumor Grades')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# 3. Feature Importance Analysis
# For cancer types
feature_importance_type = pd.DataFrame({
    'gene': X.columns,
    'importance': rf_type.feature_importances_
})
feature_importance_type = feature_importance_type.sort_values('importance', ascending=False)

# For tumor grades
feature_importance_grade = pd.DataFrame({
    'gene': X.columns,
    'importance': rf_grade.feature_importances_
})
feature_importance_grade = feature_importance_grade.sort_values('importance', ascending=False)

# Print top 10 important genes for each classification
print("\nTop 10 Important Genes for Cancer Type Classification:")
for i, row in feature_importance_type.head(10).iterrows():
    gene_symbol = gene_map.get(row['gene'], row['gene'])
    print(f"{gene_symbol}: {row['importance']:.4f}")

print("\nTop 10 Important Genes for Tumor Grade Classification:")
for i, row in feature_importance_grade.head(10).iterrows():
    gene_symbol = gene_map.get(row['gene'], row['gene'])
    print(f"{gene_symbol}: {row['importance']:.4f}")

In [ ]:
#SVM machine learning

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Data Loading and Preprocessing
print("Loading and preprocessing data...")

# Load the data
df = pd.read_csv('Breast_GSE45827p.csv')
print("First few rows of the dataframe:")
print(df.head())

# Extract features (gene expression values)
X = df.iloc[2:, 2:153].T  # Transpose to get samples as rows
X.index = df.columns[2:153]  # Set sample names as index
X.columns = df.iloc[2:, 0]  # Set probe_id as column names

# Extract labels
y_type = df.iloc[0, 2:153]
y_grade = df.iloc[1, 2:153].astype(str)

# Create a mapping of probe_id to gene symbol
gene_map = dict(zip(df.iloc[2:, 0], df.iloc[2:, 1]))

# Encode cancer types and grades
le_type = LabelEncoder()
le_grade = LabelEncoder()
y_type_encoded = le_type.fit_transform(y_type)
y_grade_encoded = le_grade.fit_transform(y_grade)

# Split the data
X_train, X_test, y_type_train, y_type_test, y_grade_train, y_grade_test = train_test_split(
    X, y_type_encoded, y_grade_encoded, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\nData preprocessing completed:")
print(f"Training set shape: {X_train_scaled.shape}")
print(f"Test set shape: {X_test_scaled.shape}")
print(f"Cancer types: {le_type.classes_}")
print(f"Tumor grades: {le_grade.classes_}")

# 2. Cancer Type Classification with SVM
print("\nTraining SVM for Cancer Type Classification...")
svm_type = SVC(kernel='linear', random_state=42)
svm_type.fit(X_train_scaled, y_type_train)

# Predict and evaluate
y_type_pred_svm = svm_type.predict(X_test_scaled)

print("\nCancer Type Classification Results (SVM):")
print("Accuracy:", accuracy_score(y_type_test, y_type_pred_svm))
print("\nClassification Report:")
print(classification_report(y_type_test, y_type_pred_svm, target_names=le_type.classes_))

# Confusion Matrix for cancer types (SVM)
plt.figure(figsize=(10, 8))
cm_type_svm = confusion_matrix(y_type_test, y_type_pred_svm)
sns.heatmap(cm_type_svm, annot=True, fmt='d', cmap='Blues',
            xticklabels=le_type.classes_,
            yticklabels=le_type.classes_)
plt.title('Confusion Matrix - Cancer Types (SVM)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# 3. Tumor Grade Classification with SVM
print("\nTraining SVM for Tumor Grade Classification...")
svm_grade = SVC(kernel='linear', random_state=42)
svm_grade.fit(X_train_scaled, y_grade_train)

# Predict and evaluate
y_grade_pred_svm = svm_grade.predict(X_test_scaled)

print("\nTumor Grade Classification Results (SVM):")
print("Accuracy:", accuracy_score(y_grade_test, y_grade_pred_svm))
print("\nClassification Report:")
print(classification_report(y_grade_test, y_grade_pred_svm, target_names=le_grade.classes_))

# Confusion Matrix for tumor grades (SVM)
plt.figure(figsize=(10, 8))
cm_grade_svm = confusion_matrix(y_grade_test, y_grade_pred_svm)
sns.heatmap(cm_grade_svm, annot=True, fmt='d', cmap='Blues',
            xticklabels=le_grade.classes_,
            yticklabels=le_grade.classes_)
plt.title('Confusion Matrix - Tumor Grades (SVM)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# 4. Feature Importance Analysis
def get_svm_feature_importance(svm_model, feature_names, class_labels):
    importance_per_class = {}
    for i, class_label in enumerate(class_labels):
        if len(class_labels) == 2 and i == 1:
            continue
        coef = svm_model.coef_[i] if len(class_labels) > 2 else svm_model.coef_[0]
        importance = pd.DataFrame({
            'gene': feature_names,
            'importance': np.abs(coef)
        })
        importance_per_class[class_label] = importance.sort_values('importance', ascending=False)
    return importance_per_class

# Get feature importance for cancer types
type_importance_svm = get_svm_feature_importance(svm_type, X.columns, le_type.classes_)

print("\nTop 10 Important Genes for Cancer Type Classification (SVM):")
for class_label, importance_df in type_importance_svm.items():
    print(f"\nFor {class_label}:")
    for i, row in importance_df.head(10).iterrows():
        gene_symbol = gene_map.get(row['gene'], row['gene'])
        print(f"{gene_symbol}: {row['importance']:.4f}")

In [ ]:
# Neural Network Analysis for Breast Cancer Gene Expression Data
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Convert labels to categorical format for Keras
y_type_train_cat = to_categorical(y_type_train)
y_type_test_cat = to_categorical(y_type_test)
y_grade_train_cat = to_categorical(y_grade_train)
y_grade_test_cat = to_categorical(y_grade_test)

# 1. Define function to create model
def create_model(input_dim, output_dim):
    model = Sequential([
        Dense(256, activation='relu', input_dim=input_dim),
        BatchNormalization(),
        Dropout(0.3),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(output_dim, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# 2. Train and evaluate cancer type classifier
print("Training Neural Network for Cancer Type Classification...")
nn_type = create_model(X_train_scaled.shape[1], len(le_type.classes_))

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history_type = nn_type.fit(X_train_scaled, y_type_train_cat,
                          validation_split=0.2,
                          epochs=100,
                          batch_size=32,
                          callbacks=[early_stopping],
                          verbose=1)

# Evaluate cancer type model
y_type_pred_nn = nn_type.predict(X_test_scaled)
y_type_pred_classes = np.argmax(y_type_pred_nn, axis=1)

print("\nCancer Type Classification Results (Neural Network):")
print("Accuracy:", accuracy_score(y_type_test, y_type_pred_classes))
print("\nClassification Report:")
print(classification_report(y_type_test, y_type_pred_classes, target_names=le_type.classes_))

# Plot training history for cancer type
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_type.history['accuracy'])
plt.plot(history_type.history['val_accuracy'])
plt.title('Cancer Type Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history_type.history['loss'])
plt.plot(history_type.history['val_loss'])
plt.title('Cancer Type Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.tight_layout()
plt.show()

# 3. Train and evaluate tumor grade classifier
print("\nTraining Neural Network for Tumor Grade Classification...")
nn_grade = create_model(X_train_scaled.shape[1], len(le_grade.classes_))

history_grade = nn_grade.fit(X_train_scaled, y_grade_train_cat,
                            validation_split=0.2,
                            epochs=100,
                            batch_size=32,
                            callbacks=[early_stopping],
                            verbose=1)

# Evaluate tumor grade model
y_grade_pred_nn = nn_grade.predict(X_test_scaled)
y_grade_pred_classes = np.argmax(y_grade_pred_nn, axis=1)

print("\nTumor Grade Classification Results (Neural Network):")
print("Accuracy:", accuracy_score(y_grade_test, y_grade_pred_classes))
print("\nClassification Report:")
print(classification_report(y_grade_test, y_grade_pred_classes, target_names=le_grade.classes_))

# Plot training history for tumor grade
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_grade.history['accuracy'])
plt.plot(history_grade.history['val_accuracy'])
plt.title('Tumor Grade Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history_grade.history['loss'])
plt.plot(history_grade.history['val_loss'])
plt.title('Tumor Grade Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.tight_layout()
plt.show()

# 4. Compare with previous models
def compare_all_models(y_true, y_pred_rf, y_pred_svm, y_pred_nn, model_type):
    print(f"\nComparison for {model_type}:")
    print(f"Random Forest Accuracy: {accuracy_score(y_true, y_pred_rf):.4f}")
    print(f"SVM Accuracy: {accuracy_score(y_true, y_pred_svm):.4f}")
    print(f"Neural Network Accuracy: {accuracy_score(y_true, y_pred_nn):.4f}")

# Compare all models
compare_all_models(y_type_test, y_type_pred, y_type_pred_svm, y_type_pred_classes, "Cancer Type Classification")
compare_all_models(y_grade_test, y_grade_pred, y_grade_pred_svm, y_grade_pred_classes, "Tumor Grade Classification")

PART 3 3. EDA_and_Comparative_machine_learning_analysis_of_ brain_cancer (Brain_GSE50161)

In [ ]:
# prompt: chaeck expression data for any N or Nan in  Brain_GSE50161g, the expression data is from the third column (column2),  third row (row2) and goes to row5466 and column131, its python rows and columns so thrid column is column2, third row is row2. make sure number of Nas or Nans are printed
import pandas as pd
import numpy as np
# Assuming Brain_GSE50161g is your DataFrame

Brain_GSE50161g = pd.read_csv("Brain_GSE50161g.csv", header=None)
print(Brain_GSE50161g)

nan_count = Brain_GSE50161g.iloc[2:5467, 2:132].isnull().sum().sum()
n_count = (Brain_GSE50161g.iloc[2:5467, 2:132] == 'N').sum().sum()


print(f"Number of NaN values: {nan_count}")
print(f"Number of 'N' values: {n_count}")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# Load the CSV file into a DataFrame
Brain_GSE50161g = pd.read_csv('Brain_GSE50161g.csv')

# 1. Print dimensions of the data
print(f"Dimensions of the data: {Brain_GSE50161g.shape}")

# 2. Extract relevant parts of the data
sample_ids6 = Brain_GSE50161g.iloc[0, 2:132].values    # Sample IDs in row 1 (Python index 0)
sample_types6 = Brain_GSE50161g.iloc[1, 2:132].values  # Sample types in row 2 (Python index 1)
sample_grades6 = Brain_GSE50161g.iloc[2, 2:132].values  # Sample grades in row 3 (Python index 2)
probe_ids6 = Brain_GSE50161g.iloc[3:, 0].values         # Probe IDs starting from row 4 (Python index 3)
gene_symbols6 = Brain_GSE50161g.iloc[3:, 1].values      # Gene symbols starting from row 4
gene_expression6 = Brain_GSE50161g.iloc[3:, 2:132].T    # Transpose to have samples as rows

# 3. Print the first few sample types and grades
print("First few sample types (Sample IDs):")
print(sample_ids6[:5])
print("First few sample grades:")
print(sample_types6[:5])

# 4. Set row and column names for the gene expression matrix
gene_expression6.columns = probe_ids6
gene_expression6.index = sample_ids6

# 5. Convert gene expression data to numeric
gene_expression6 = gene_expression6.apply(pd.to_numeric)

# 6. Perform PCA
pca = PCA()
pca_result6 = pca.fit_transform(gene_expression6)

# 7. Extract PC1 and PC2
pc_data6 = pd.DataFrame({
    'PC1': pca_result6[:, 0],
    'PC2': pca_result6[:, 1],
    'Type': sample_types6,
    'Grade': sample_grades6
})

# 8. Print the structure of pc_data6
print("Structure of pc_data6:")
print(pc_data6.info())

# 9. Calculate variance explained
var_explained6 = pca.explained_variance_ratio_

# 10. Plot PCA results colored by Type
plt.figure(figsize=(10, 6))
sns.scatterplot(x='PC1', y='PC2', hue='Type', data=pc_data6, s=100, alpha=0.7)
plt.title(f'PCA of Brain Cancer Gene Expression Data (GSE50161) by Type')
plt.xlabel(f'PC1 ({var_explained6[0] * 100:.2f}% variance explained)')
plt.ylabel(f'PC2 ({var_explained6[1] * 100:.2f}% variance explained)')
plt.legend(title='Sample Type')
plt.grid(True)
plt.show()

# 11. Plot PCA results colored by Grade
plt.figure(figsize=(10, 6))
sns.scatterplot(x='PC1', y='PC2', hue='Grade', data=pc_data6, s=100, alpha=0.7)
plt.title(f'PCA of Brain Cancer Gene Expression Data (GSE50161) by Grade')
plt.xlabel(f'PC1 ({var_explained6[0] * 100:.2f}% variance explained)')
plt.ylabel(f'PC2 ({var_explained6[1] * 100:.2f}% variance explained)')
plt.legend(title='Sample Grade')
plt.grid(True)
plt.show()

# 12. Print summary of PCA
print("PCA Summary:")
print(f"Variance explained by first two components: PC1 = {var_explained6[0] * 100:.2f}%, PC2 = {var_explained6[1] * 100:.2f}%")

# 13. Display the first few rows of the PC data
print("First few rows of PC data:")
print(pc_data6.head())

In [ ]:
# Hierarchical Clustering with Heatmap of AFFX and controls to judge whether data is normalised
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data
df = pd.read_csv('Brain_GSE50161g.csv')

# Determine the actual dimensions of your data
num_rows = df.shape[0]
num_cols = df.shape[1]

# Extract sample data (adjust indices based on your actual data structure)
# Assuming the data starts from row 3 and column 3
sample_data = df.iloc[2:num_rows, 2:num_cols].astype(float)

# Extract probe IDs and gene symbols
probe_ids = df.iloc[2:num_rows, 0]
gene_symbols = df.iloc[2:num_rows, 1]

# Extract sample names, types, and grades
sample_names = df.columns[2:num_cols]
sample_types = df.iloc[0, 2:num_cols]
sample_grades = df.iloc[1, 2:num_cols]

# Filter for AFFX probes
affx_mask = probe_ids.str.contains('AFFX', case=False, na=False)
affx_data = sample_data[affx_mask]
affx_probe_ids = probe_ids[affx_mask]

# Create a DataFrame with AFFX probe data
affx_df = pd.DataFrame(affx_data.values, index=affx_probe_ids, columns=sample_names)

# Create the heatmap
plt.figure(figsize=(20, 10))
sns.heatmap(affx_df, cmap='coolwarm', center=0, cbar_kws={'label': 'Expression Value'})
plt.title('Heatmap of AFFX Probes Across Brain Samples')
plt.xlabel('Samples')
plt.ylabel('AFFX Probe IDs')
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Print some statistics
print(f"Total number of AFFX probes: {len(affx_probe_ids)}")
print("\nFirst 10 AFFX probe IDs:")
print(affx_probe_ids[:10].tolist())

# Calculate and print average expression for each AFFX probe
avg_expression = affx_df.mean(axis=1).sort_values(ascending=False)
print("\nTop 10 AFFX probes by average expression:")
print(avg_expression.head(10))

print("\nBottom 10 AFFX probes by average expression:")
print(avg_expression.tail(10))

THE MACHINE LEARNING

In [ ]:
# Corrected Data Preprocessing for Brain Cancer Gene Expression Analysis
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

# Load the data
df = pd.read_csv('Brain_GSE50161g.csv')

# Extract features (gene expression values)
# Adjusting the row indices to start one row lower
X = df.iloc[3:, 2:153].T  # Transpose to get samples as rows
X.index = df.columns[2:153]  # Set sample names as index
X.columns = df.iloc[3:, 0]  # Set probe_id as column names

# Extract labels - shifting down by one row
y_type = df.iloc[1, 2:153]  # Changed from 0 to 1
y_grade = df.iloc[2, 2:153].astype(str)  # Changed from 1 to 2

# Create a mapping of probe_id to gene symbol - adjusted for the row shift
gene_map = dict(zip(df.iloc[3:, 0], df.iloc[3:, 1]))

# Encode cancer types and grades
le_type = LabelEncoder()
le_grade = LabelEncoder()
y_type_encoded = le_type.fit_transform(y_type)
y_grade_encoded = le_grade.fit_transform(y_grade)

# Split the data
X_train, X_test, y_type_train, y_type_test, y_grade_train, y_grade_test = train_test_split(
    X, y_type_encoded, y_grade_encoded, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Print information about the processed data
print("Data preprocessed and split into training and test sets.")
print(f"Training set shape: {X_train_scaled.shape}")
print(f"Test set shape: {X_test_scaled.shape}")
print(f"Cancer types: {le_type.classes_}")
print(f"Tumor grades: {le_grade.classes_}")

# Optional: Print first few rows of cancer types and grades to verify
print("\nFirst few cancer types:")
print(y_type[:5])
print("\nFirst few grades:")
print(y_grade[:5])

In [ ]:
# Machine Learning Analysis for Brain Cancer Gene Expression Data
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Using the preprocessed data from previous code
# Assuming we have: X_train_scaled, X_test_scaled, y_type_train, y_type_test, y_grade_train, y_grade_test

# 1. Train and evaluate cancer type classifier
rf_type = RandomForestClassifier(n_estimators=100, random_state=42)
rf_type.fit(X_train_scaled, y_type_train)

# Predict and evaluate
y_type_pred = rf_type.predict(X_test_scaled)

print("Cancer Type Classification Results:")
print("Accuracy:", accuracy_score(y_type_test, y_type_pred))
print("\nClassification Report:")
print(classification_report(y_type_test, y_type_pred, target_names=le_type.classes_))

# Confusion Matrix for cancer types
plt.figure(figsize=(10, 8))
cm_type = confusion_matrix(y_type_test, y_type_pred)
sns.heatmap(cm_type, annot=True, fmt='d', cmap='Blues',
            xticklabels=le_type.classes_,
            yticklabels=le_type.classes_)
plt.title('Confusion Matrix - Cancer Types')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# 2. Train and evaluate tumor grade classifier
rf_grade = RandomForestClassifier(n_estimators=100, random_state=42)
rf_grade.fit(X_train_scaled, y_grade_train)

# Predict and evaluate
y_grade_pred = rf_grade.predict(X_test_scaled)

print("\nTumor Grade Classification Results:")
print("Accuracy:", accuracy_score(y_grade_test, y_grade_pred))
print("\nClassification Report:")
print(classification_report(y_grade_test, y_grade_pred, target_names=le_grade.classes_))

# Confusion Matrix for tumor grades
plt.figure(figsize=(10, 8))
cm_grade = confusion_matrix(y_grade_test, y_grade_pred)
sns.heatmap(cm_grade, annot=True, fmt='d', cmap='Blues',
            xticklabels=le_grade.classes_,
            yticklabels=le_grade.classes_)
plt.title('Confusion Matrix - Tumor Grades')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# 3. Feature Importance Analysis
# For cancer types
feature_importance_type = pd.DataFrame({
    'gene': X.columns,
    'importance': rf_type.feature_importances_
})
feature_importance_type = feature_importance_type.sort_values('importance', ascending=False)

# For tumor grades
feature_importance_grade = pd.DataFrame({
    'gene': X.columns,
    'importance': rf_grade.feature_importances_
})
feature_importance_grade = feature_importance_grade.sort_values('importance', ascending=False)

# Print top 10 important genes for each classification
print("\nTop 10 Important Genes for Cancer Type Classification:")
for i, row in feature_importance_type.head(10).iterrows():
    gene_symbol = gene_map.get(row['gene'], row['gene'])
    print(f"{gene_symbol}: {row['importance']:.4f}")

print("\nTop 10 Important Genes for Tumor Grade Classification:")
for i, row in feature_importance_grade.head(10).iterrows():
    gene_symbol = gene_map.get(row['gene'], row['gene'])
    print(f"{gene_symbol}: {row['importance']:.4f}")

In [ ]:
# SVM Analysis for Brain Cancer Gene Expression Data
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Cancer Type Classification with SVM
print("Training SVM for Cancer Type Classification...")
svm_type = SVC(kernel='linear', random_state=42)
svm_type.fit(X_train_scaled, y_type_train)

# Predict and evaluate
y_type_pred_svm = svm_type.predict(X_test_scaled)

print("\nCancer Type Classification Results (SVM):")
print("Accuracy:", accuracy_score(y_type_test, y_type_pred_svm))
print("\nClassification Report:")
print(classification_report(y_type_test, y_type_pred_svm, target_names=le_type.classes_))

# Confusion Matrix for cancer types (SVM)
plt.figure(figsize=(10, 8))
cm_type_svm = confusion_matrix(y_type_test, y_type_pred_svm)
sns.heatmap(cm_type_svm, annot=True, fmt='d', cmap='Blues',
            xticklabels=le_type.classes_,
            yticklabels=le_type.classes_)
plt.title('Confusion Matrix - Cancer Types (SVM)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# 2. Tumor Grade Classification with SVM
print("\nTraining SVM for Tumor Grade Classification...")
svm_grade = SVC(kernel='linear', random_state=42)
svm_grade.fit(X_train_scaled, y_grade_train)

# Predict and evaluate
y_grade_pred_svm = svm_grade.predict(X_test_scaled)

print("\nTumor Grade Classification Results (SVM):")
print("Accuracy:", accuracy_score(y_grade_test, y_grade_pred_svm))
print("\nClassification Report:")
print(classification_report(y_grade_test, y_grade_pred_svm, target_names=le_grade.classes_))

# Confusion Matrix for tumor grades (SVM)
plt.figure(figsize=(10, 8))
cm_grade_svm = confusion_matrix(y_grade_test, y_grade_pred_svm)
sns.heatmap(cm_grade_svm, annot=True, fmt='d', cmap='Blues',
            xticklabels=le_grade.classes_,
            yticklabels=le_grade.classes_)
plt.title('Confusion Matrix - Tumor Grades (SVM)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# 3. Compare SVM vs Random Forest
def compare_models(y_true, y_pred_rf, y_pred_svm, model_type):
    print(f"\nComparison for {model_type}:")
    print(f"Random Forest Accuracy: {accuracy_score(y_true, y_pred_rf):.4f}")
    print(f"SVM Accuracy: {accuracy_score(y_true, y_pred_svm):.4f}")

    # Creating a DataFrame for easier comparison
    comparison_df = pd.DataFrame({
        'True Labels': y_true,
        'RF Predictions': y_pred_rf,
        'SVM Predictions': y_pred_svm
    })

    # Find where models disagree
    disagreements = comparison_df[comparison_df['RF Predictions'] != comparison_df['SVM Predictions']]
    print(f"\nNumber of disagreements between models: {len(disagreements)}")

# Compare models for both cancer type and grade
compare_models(y_type_test, y_type_pred, y_type_pred_svm, "Cancer Type Classification")
compare_models(y_grade_test, y_grade_pred, y_grade_pred_svm, "Tumor Grade Classification")

# 4. Feature Importance for SVM
# For linear SVM, feature importance can be derived from the coefficients
def get_svm_feature_importance(svm_model, feature_names, class_labels):
    importance_per_class = {}
    for i, class_label in enumerate(class_labels):
        if len(class_labels) == 2 and i == 1:
            # For binary classification, we only have one set of coefficients
            continue
        coef = svm_model.coef_[i] if len(class_labels) > 2 else svm_model.coef_[0]
        importance = pd.DataFrame({
            'gene': feature_names,
            'importance': np.abs(coef)
        })
        importance_per_class[class_label] = importance.sort_values('importance', ascending=False)
    return importance_per_class

# Get feature importance for cancer types
type_importance_svm = get_svm_feature_importance(svm_type, X.columns, le_type.classes_)

print("\nTop 10 Important Genes for Cancer Type Classification (SVM):")
for class_label, importance_df in type_importance_svm.items():
    print(f"\nFor {class_label}:")
    for i, row in importance_df.head(10).iterrows():
        gene_symbol = gene_map.get(row['gene'], row['gene'])
        print(f"{gene_symbol}: {row['importance']:.4f}")

In [ ]:
# Neural Network Analysis for Brain Cancer Gene Expression Data
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Convert labels to categorical format for Keras
y_type_train_cat = to_categorical(y_type_train)
y_type_test_cat = to_categorical(y_type_test)
y_grade_train_cat = to_categorical(y_grade_train)
y_grade_test_cat = to_categorical(y_grade_test)

# 1. Define function to create model
def create_model(input_dim, output_dim):
    model = Sequential([
        Dense(256, activation='relu', input_dim=input_dim),
        BatchNormalization(),
        Dropout(0.3),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(output_dim, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# 2. Train and evaluate cancer type classifier
print("Training Neural Network for Cancer Type Classification...")
nn_type = create_model(X_train_scaled.shape[1], len(le_type.classes_))

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history_type = nn_type.fit(X_train_scaled, y_type_train_cat,
                          validation_split=0.2,
                          epochs=100,
                          batch_size=32,
                          callbacks=[early_stopping],
                          verbose=1)

# Evaluate cancer type model
y_type_pred_nn = nn_type.predict(X_test_scaled)
y_type_pred_classes = np.argmax(y_type_pred_nn, axis=1)

print("\nCancer Type Classification Results (Neural Network):")
print("Accuracy:", accuracy_score(y_type_test, y_type_pred_classes))
print("\nClassification Report:")
print(classification_report(y_type_test, y_type_pred_classes, target_names=le_type.classes_))

# Plot training history for cancer type
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_type.history['accuracy'])
plt.plot(history_type.history['val_accuracy'])
plt.title('Cancer Type Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history_type.history['loss'])
plt.plot(history_type.history['val_loss'])
plt.title('Cancer Type Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.tight_layout()
plt.show()

# 3. Train and evaluate tumor grade classifier
print("\nTraining Neural Network for Tumor Grade Classification...")
nn_grade = create_model(X_train_scaled.shape[1], len(le_grade.classes_))

history_grade = nn_grade.fit(X_train_scaled, y_grade_train_cat,
                            validation_split=0.2,
                            epochs=100,
                            batch_size=32,
                            callbacks=[early_stopping],
                            verbose=1)

# Evaluate tumor grade model
y_grade_pred_nn = nn_grade.predict(X_test_scaled)
y_grade_pred_classes = np.argmax(y_grade_pred_nn, axis=1)

print("\nTumor Grade Classification Results (Neural Network):")
print("Accuracy:", accuracy_score(y_grade_test, y_grade_pred_classes))
print("\nClassification Report:")
print(classification_report(y_grade_test, y_grade_pred_classes, target_names=le_grade.classes_))

# Plot training history for tumor grade
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_grade.history['accuracy'])
plt.plot(history_grade.history['val_accuracy'])
plt.title('Tumor Grade Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history_grade.history['loss'])
plt.plot(history_grade.history['val_loss'])
plt.title('Tumor Grade Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.tight_layout()
plt.show()

# 4. Compare with previous models
def compare_all_models(y_true, y_pred_rf, y_pred_svm, y_pred_nn, model_type):
    print(f"\nComparison for {model_type}:")
    print(f"Random Forest Accuracy: {accuracy_score(y_true, y_pred_rf):.4f}")
    print(f"SVM Accuracy: {accuracy_score(y_true, y_pred_svm):.4f}")
    print(f"Neural Network Accuracy: {accuracy_score(y_true, y_pred_nn):.4f}")

# Compare all models
compare_all_models(y_type_test, y_type_pred, y_type_pred_svm, y_type_pred_classes, "Cancer Type Classification")
compare_all_models(y_grade_test, y_grade_pred, y_grade_pred_svm, y_grade_pred_classes, "Tumor Grade Classification")

MULTI CANCER SECTION

In [ ]:
#section1 his script processes two datasets, Breast_GSE45827q and Brain_GSE50161h, to prepare them for analysis. First, it reads the breast cancer dataset from a CSV file and applies quantile transformation to the numeric columns, normalizing their distributions using the inverse of the cumulative distribution function (CDF) from the normal distribution. After transforming the numeric data, it reassigns the original column names to the first row of the DataFrame for clarity. Lastly, it reads the brain cancer
# dataset from another CSV file, preserving the first row as column titles, and prints the resulting DataFrames for inspection.

import pandas as pd
import numpy as np
from scipy.stats import norm

# Generate Breast_GSE45827q and transform columns
Breast_GSE45827q = pd.read_csv("Breast_GSE45827p.csv", header=0)

# Get numeric columns and apply Q transformation
numeric_cols = Breast_GSE45827q.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    values = Breast_GSE45827q[col].values
    ranks = np.argsort(np.argsort(values))
    n = len(values)
    quantiles = (ranks + 1) / (n + 1)
    Breast_GSE45827q[col] = norm.ppf(quantiles)

# Move column names to first row
old_columns = Breast_GSE45827q.columns.tolist()
Breast_GSE45827q.columns = range(len(old_columns))
Breast_GSE45827q.loc[-1] = old_columns
Breast_GSE45827q.index = Breast_GSE45827q.index + 1
Breast_GSE45827q = Breast_GSE45827q.sort_index()
print(Breast_GSE45827q)
import pandas as pd

# Open Brain_GSE50161g.csv as Brain_GSE50161h dataframe
import pandas as pd

# Open Brain_GSE50161g.csv as Brain_GSE50161h dataframe with the first row as column titles
Brain_GSE50161h = pd.read_csv("Brain_GSE50161g.csv", header=0)
print(Brain_GSE50161h)

In [ ]:
#Section2
# Create a new dataframe Brain_GSE50161i from Brain_GSE50161h
Brain_GSE50161i = Brain_GSE50161h.copy()

# Change the column names of Brain_GSE50161i to start with 0
Brain_GSE50161i.columns = range(0, Brain_GSE50161i.shape[1])

# Print the new dataframe to verify
print(Brain_GSE50161i)


This script defines a function to merge two microarray datasets—one for brain cancer and one for breast cancer—into a single DataFrame. The function merge_microarray_data performs the following steps:

Data Preparation: It resets the index for both input DataFrames (brain_df and breast_df) to ensure clean indexing. It then extracts the first three rows as metadata and the remaining rows as probe data for both datasets.

Metadata Combination: The function creates a new DataFrame to combine the metadata from both datasets. It retains the first two columns from the breast metadata and appends the corresponding columns from the brain metadata.

Probe Data Combination: Similarly, it initializes a DataFrame for the probe data, combining both the breast and brain probe data while preserving their original structure.

Final Merge: The metadata and probe data are concatenated vertically to form the final merged DataFrame, breastandbrain.

Verification and Output: The script prints the shapes of the original DataFrames and the merged DataFrame for verification. It checks if the total number of columns matches the expected count and outputs the merged DataFrame to a CSV file named breastandbrain.csv.

The overall goal is to consolidate the datasets for subsequent analysis, ensuring all relevant information is retained and correctly formatted.

In [ ]:
#section3

import pandas as pd
import numpy as np

def merge_microarray_data(brain_df, breast_df):
    # Create new clean indices for both dataframes
    brain_clean = brain_df.copy().reset_index(drop=True)
    breast_clean = breast_df.copy().reset_index(drop=True)

    # Extract metadata (first 3 rows)
    breast_metadata = breast_clean.iloc[:3].copy()
    brain_metadata = brain_clean.iloc[:3].copy()

    # Extract probe data (remaining rows)
    breast_probes = breast_clean.iloc[3:].copy().reset_index(drop=True)
    brain_probes = brain_clean.iloc[3:].copy().reset_index(drop=True)

    # Combine metadata
    combined_metadata = pd.DataFrame(index=[0, 1, 2])

    # First two columns from breast data
    combined_metadata[0] = breast_metadata[0]
    combined_metadata[1] = breast_metadata[1]

    # Add breast samples
    for i in range(2, breast_clean.shape[1]):
        combined_metadata[i] = breast_metadata[i]

    # Add brain samples
    for i in range(2, brain_clean.shape[1]):
        new_col = i + breast_clean.shape[1] - 2
        combined_metadata[new_col] = brain_metadata[i]

    # Combine probe data
    combined_probes = pd.DataFrame()

    # First two columns from breast data
    combined_probes[0] = breast_probes[0]
    combined_probes[1] = breast_probes[1]

    # Add breast probe data
    for i in range(2, breast_clean.shape[1]):
        combined_probes[i] = breast_probes[i]

    # Add brain probe data
    for i in range(2, brain_clean.shape[1]):
        new_col = i + breast_clean.shape[1] - 2
        combined_probes[new_col] = brain_probes[i]

    # Combine metadata and probe data
    breastandbrain = pd.concat([combined_metadata, combined_probes], axis=0).reset_index(drop=True)

    return breastandbrain

# Print info about the dataframes
print(f"Brain_GSE50161i shape: {Brain_GSE50161i.shape}")
print(f"Breast_GSE45827q shape: {Breast_GSE45827q.shape}")

# Execute the merge
breastandbrain = merge_microarray_data(Brain_GSE50161i, Breast_GSE45827q)

print(f"\nShape of merged dataframe: {breastandbrain.shape}")
print("\nFirst few rows of the merged dataframe:")
print(breastandbrain.iloc[:5, :10])  # Show first 5 rows and 10 columns

# Verification
print("\nVerification:")
print(f"Total number of columns: {breastandbrain.shape[1]}")
print(f"Expected number of columns: {Brain_GSE50161i.shape[1] + Breast_GSE45827q.shape[1] - 2}")
print(f"Number of rows: {breastandbrain.shape[0]}")

breastandbrain.to_csv("breastandbrain.csv", index=False)

In [ ]:
#Section5 Check to see if normalisation is needed by checkin control gene heatmap
# Hierarchical Clustering with Heatmap of AFFX and controls
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data
df = pd.read_csv('breastandbrain.csv')

# Determine the actual dimensions of your data
num_rows = df.shape[0]
num_cols = df.shape[1]

# Extract sample data (adjust indices based on your actual data structure)
# Assuming the data starts from row 3 and column 3
sample_data = df.iloc[2:num_rows, 2:num_cols].astype(float)

# Extract probe IDs and gene symbols
probe_ids = df.iloc[2:num_rows, 0]
gene_symbols = df.iloc[2:num_rows, 1]

# Extract sample names, types, and grades
sample_names = df.columns[2:num_cols]
sample_types = df.iloc[0, 2:num_cols]
sample_grades = df.iloc[1, 2:num_cols]

# Filter for AFFX probes
affx_mask = probe_ids.str.contains('AFFX', case=False, na=False)
affx_data = sample_data[affx_mask]
affx_probe_ids = probe_ids[affx_mask]

# Create a DataFrame with AFFX probe data
affx_df = pd.DataFrame(affx_data.values, index=affx_probe_ids, columns=sample_names)

# Create the heatmap
plt.figure(figsize=(20, 10))
sns.heatmap(affx_df, cmap='coolwarm', center=0, cbar_kws={'label': 'Expression Value'})
plt.title('Heatmap of AFFX Probes Across Brain Samples')
plt.xlabel('Samples')
plt.ylabel('AFFX Probe IDs')
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Print some statistics
print(f"Total number of AFFX probes: {len(affx_probe_ids)}")
print("\nFirst 10 AFFX probe IDs:")
print(affx_probe_ids[:10].tolist())

# Calculate and print average expression for each AFFX probe
avg_expression = affx_df.mean(axis=1).sort_values(ascending=False)
print("\nTop 10 AFFX probes by average expression:")
print(avg_expression.head(10))

print("\nBottom 10 AFFX probes by average expression:")
print(avg_expression.tail(10))

In [ ]:
#section6
#Normalisation of data acros the 2 combined dataframes
import pandas as pd
import numpy as np
from sklearn.preprocessing import QuantileTransformer

# Load the data, with low_memory=False to avoid mixed-type warnings
df = pd.read_csv('breastandbrain.csv', low_memory=False)

# Separate the first three rows (metadata) and the rest of the data
metadata_rows = df.iloc[:3]

# Keep the first two columns separate
first_two_columns = df.iloc[3:, :2]

# Expression data starts from column 3 (index 2)
expression_data = df.iloc[3:, 2:]

# Convert expression data to numeric (coerce errors to NaN)
expression_data = expression_data.apply(pd.to_numeric, errors='coerce')

# Create a QuantileTransformer object
transformer = QuantileTransformer(output_distribution='uniform', random_state=0)

# Fit the transformer to the expression data (ignoring NaN values)
normalized_data = transformer.fit_transform(expression_data)

# Create a new DataFrame for the normalized data
normalized_df = pd.DataFrame(normalized_data, columns=expression_data.columns)

# Combine the first two columns with the normalized data
combined_df = pd.concat([first_two_columns.reset_index(drop=True),
                         normalized_df], axis=1)

# Combine the metadata with the combined data
final_df = pd.concat([metadata_rows, combined_df], ignore_index=True)

# Save the output as 'breastandbrain_normalized.csv'
final_df.to_csv('breastandbrain_normalized.csv', index=False)
print("Normalization complete and saved as 'breastandbrain_normalized.csv'")

In [ ]:
#Section 7 Detect NAz in the dataframe, only in col['1']
# Detect Nan vales in data, Assuming 'df' is your DataFrame
nan_columns = final_df.columns[final_df.isna().any()].tolist() # Changed df to final_df

print(f"Columns with NaN values: {nan_columns}")

In [ ]:
# Hierarchical Clustering with Heatmap of AFFX and controls- chec on normalisation process
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data, with low_memory=False to avoid mixed-type warnings
df = pd.read_csv('breastandbrain_normalized.csv', header=None, low_memory=False)
print(df.head())

# Separate metadata (Probe IDs and Gene Symbols) and expression data
probe_ids = df.iloc[3:, 0]  # Probe IDs start from row 4 (index 3), column 1 (index 0)
gene_symbols = df.iloc[3:, 1]  # Gene Symbols in column 2 (index 1)
expression_data = df.iloc[3:, 2:]  # Expression data starts from column 3 (index 2)

# Convert expression data to numeric (coerce errors to NaN)
expression_data = expression_data.apply(pd.to_numeric, errors='coerce')

# Filter for AFFX probes in the Probe ID column
affx_mask = probe_ids.str.contains('AFFX', case=False, na=False)
affx_expression_data = expression_data[affx_mask]
affx_probe_ids = probe_ids[affx_mask]
affx_gene_symbols = gene_symbols[affx_mask]

# Create a DataFrame with AFFX probe data
affx_df = pd.DataFrame(affx_expression_data.values,
                       index=pd.MultiIndex.from_arrays([affx_probe_ids, affx_gene_symbols],
                                                       names=['Probe_ID', 'Gene_Symbol']),
                       columns=df.iloc[0, 2:])  # Sample names in row 1 (index 0)

# Plot the heatmap for AFFX probes
plt.figure(figsize=(20, 10))
sns.heatmap(affx_df, cmap='coolwarm', center=0, cbar_kws={'label': 'Expression Value'})
plt.title('Heatmap of AFFX Probes Across Brain and Breast Samples')
plt.xlabel('Samples')
plt.ylabel('AFFX Probe IDs')
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Print some statistics
print(f"Total number of AFFX probes: {len(affx_probe_ids)}")
print("\nFirst 10 AFFX probe IDs:")
print(affx_probe_ids[:10].tolist())

# Calculate and print average expression for each AFFX probe
avg_expression = affx_df.mean(axis=1).sort_values(ascending=False)
print("\nTop 10 AFFX probes by average expression:")
print(avg_expression.head(10))

print("\nBottom 10 AFFX probes by average expression:")
print(avg_expression.tail(10))

In [ ]:
#PCA analysis with and without normalization for comparison
#PCA ANALYSIS WITHOUT NORMALISATION
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# Load the CSV file into a DataFrame
breastandbrain = pd.read_csv('breastandbrain.csv')

# 1. Print dimensions of the data
print(f"Dimensions of the data: {breastandbrain.shape}")

# 2. Extract relevant parts of the data
sample_ids6 = breastandbrain.iloc[0, 2:283].values    # Sample IDs in row 1 (Python index 0)
sample_types6 = breastandbrain.iloc[1, 2:283].values  # Sample types in row 2 (Python index 1)
sample_grades6 = breastandbrain.iloc[2, 2:283].values  # Sample grades in row 3 (Python index 2)
probe_ids6 = breastandbrain.iloc[3:, 0].values         # Probe IDs starting from row 4 (Python index 3)
gene_symbols6 = breastandbrain.iloc[3:, 1].values      # Gene symbols starting from row 4
gene_expression6 = breastandbrain.iloc[3:, 2:283].T    # Transpose to have samples as rows

# 3. Print the first few sample types and grades
print("First few sample types (Sample IDs):")
print(sample_ids6[:5])
print("First few sample grades:")
print(sample_types6[:5])

# 4. Set row and column names for the gene expression matrix
gene_expression6.columns = probe_ids6
gene_expression6.index = sample_ids6

# 5. Convert gene expression data to numeric
gene_expression6 = gene_expression6.apply(pd.to_numeric)

# 6. Perform PCA
pca = PCA()
pca_result6 = pca.fit_transform(gene_expression6)

# 7. Extract PC1 and PC2
pc_data6 = pd.DataFrame({
    'PC1': pca_result6[:, 0],
    'PC2': pca_result6[:, 1],
    'Type': sample_types6,
    'Grade': sample_grades6
})

# 8. Print the structure of pc_data6
print("Structure of pc_data6:")
print(pc_data6.info())

# 9. Calculate variance explained
var_explained6 = pca.explained_variance_ratio_

# 10. Plot PCA results colored by Type
plt.figure(figsize=(10, 6))
sns.scatterplot(x='PC1', y='PC2', hue='Type', data=pc_data6, s=100, alpha=0.7)
plt.title(f'PCA of Breast and Brain Cancer Gene Expression Data by Type')
plt.xlabel(f'PC1 ({var_explained6[0] * 100:.2f}% variance explained)')
plt.ylabel(f'PC2 ({var_explained6[1] * 100:.2f}% variance explained)')
plt.legend(title='Sample Type')
plt.grid(True)
plt.show()

# 11. Plot PCA results colored by grade
plt.figure(figsize=(10, 6))
sns.scatterplot(x='PC1', y='PC2', hue='Grade', data=pc_data6, s=100, alpha=0.7)
plt.title(f'PCA of Breast and Brain Cancer Gene Expression Data by Grade')
plt.xlabel(f'PC1 ({var_explained6[0] * 100:.2f}% variance explained)')
plt.ylabel(f'PC2 ({var_explained6[1] * 100:.2f}% variance explained)')
plt.legend(title='Sample Type')
plt.grid(True)
plt.show()

# 12. Print summary of PCA
print("PCA Summary:")
print(f"Variance explained by first two components: PC1 = {var_explained6[0] * 100:.2f}%, PC2 = {var_explained6[1] * 100:.2f}%")

# 13. Display the first few rows of the PC data
print("First few rows of PC data:")
print(pc_data6.head())



In [ ]:
#PCA ANALYSIS WITHOUT NORMALISATION
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# Load the CSV file into a DataFrame
breastandbrain = pd.read_csv('breastandbrain.csv')

# 1. Print dimensions of the data
print(f"Dimensions of the data: {breastandbrain.shape}")

# 2. Extract relevant parts of the data
sample_ids6 = breastandbrain.iloc[0, 2:283].values    # Sample IDs in row 1 (Python index 0)
sample_types6 = breastandbrain.iloc[1, 2:283].values  # Sample types in row 2 (Python index 1)
sample_grades6 = breastandbrain.iloc[2, 2:283].values  # Sample grades in row 3 (Python index 2)
probe_ids6 = breastandbrain.iloc[3:, 0].values         # Probe IDs starting from row 4 (Python index 3)
gene_symbols6 = breastandbrain.iloc[3:, 1].values      # Gene symbols starting from row 4
gene_expression6 = breastandbrain.iloc[3:, 2:283].T    # Transpose to have samples as rows

# 3. Print the first few sample types and grades
print("First few sample types (Sample IDs):")
print(sample_ids6[:5])
print("First few sample grades:")
print(sample_types6[:5])

# 4. Set row and column names for the gene expression matrix
gene_expression6.columns = probe_ids6
gene_expression6.index = sample_ids6

# 5. Convert gene expression data to numeric
gene_expression6 = gene_expression6.apply(pd.to_numeric)

# 6. Perform PCA
pca = PCA()
pca_result6 = pca.fit_transform(gene_expression6)

# 7. Extract PC1 and PC2
pc_data6 = pd.DataFrame({
    'PC1': pca_result6[:, 0],
    'PC2': pca_result6[:, 1],
    'Type': sample_types6,
    'Grade': sample_grades6
})

# 8. Print the structure of pc_data6
print("Structure of pc_data6:")
print(pc_data6.info())

# 9. Calculate variance explained
var_explained6 = pca.explained_variance_ratio_

# 10. Plot PCA results colored by Type
plt.figure(figsize=(10, 6))
sns.scatterplot(x='PC1', y='PC2', hue='Type', data=pc_data6, s=100, alpha=0.7)
plt.title(f'PCA of Breast and Brain Cancer Gene Expression Data by Type')
plt.xlabel(f'PC1 ({var_explained6[0] * 100:.2f}% variance explained)')
plt.ylabel(f'PC2 ({var_explained6[1] * 100:.2f}% variance explained)')
plt.legend(title='Sample Type')
plt.grid(True)
plt.show()

# 11. Plot PCA results colored by grade
plt.figure(figsize=(10, 6))
sns.scatterplot(x='PC1', y='PC2', hue='Grade', data=pc_data6, s=100, alpha=0.7)
plt.title(f'PCA of Breast and Brain Cancer Gene Expression Data by Grade')
plt.xlabel(f'PC1 ({var_explained6[0] * 100:.2f}% variance explained)')
plt.ylabel(f'PC2 ({var_explained6[1] * 100:.2f}% variance explained)')
plt.legend(title='Sample Type')
plt.grid(True)
plt.show()

# 12. Print summary of PCA
print("PCA Summary:")
print(f"Variance explained by first two components: PC1 = {var_explained6[0] * 100:.2f}%, PC2 = {var_explained6[1] * 100:.2f}%")

# 13. Display the first few rows of the PC data
print("First few rows of PC data:")
print(pc_data6.head())

In [ ]:
#section 11
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

# Load the data
df = final_df
print(df.head()) # Use head() to display the first few rows of the dataframe

In [ ]:
#The machine learning model implemented is a Support Vector Machine (SVM) with a linear kernel, designed for classifying breast cancer types and tumor grades from gene expression data. The model leverages class weights to address class imbalance, ensuring that minority classes are given more importance during training. Performance is evaluated using metrics such as accuracy, classification reports,
#and confusion matrices, providing insights into the model's effectiveness across different classes.
#mode1 1
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Data Loading and Preprocessing

breastandbrain1 = final_df
# Load the data
df = breastandbrain1
print("First few rows of the dataframe:")
print(df.head())

# Extract features (gene expression values)
X = df.iloc[3:, 2:283].T  # Transpose to get samples as rows and genes as columns
X.index = df.iloc[0, 2:283]  # Set Sample_IDs as index
X.columns = df.iloc[3:, 0]  # Set Probe_IDs as column names

# Extract labels
y_type = df.iloc[1, 2:283]  # Sample type is in the second row
y_grade = df.iloc[2, 2:283].astype(str)  # Grade is in the third row

# Create a mapping of probe_id to gene symbol (if needed)
gene_map = dict(zip(df.iloc[3:, 0], df.iloc[3:, 1]))

# Encode cancer types and grades
le_type = LabelEncoder()
le_grade = LabelEncoder()
y_type_encoded = le_type.fit_transform(y_type)
y_grade_encoded = le_grade.fit_transform(y_grade)

# Split the data
X_train, X_test, y_type_train, y_type_test, y_grade_train, y_grade_test = train_test_split(
    X, y_type_encoded, y_grade_encoded, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\nData preprocessing completed:")
print(f"Training set shape: {X_train_scaled.shape}")
print(f"Test set shape: {X_test_scaled.shape}")
print(f"Cancer types: {le_type.classes_}")
print(f"Tumor grades: {le_grade.classes_}")

# 2. Cancer Type Classification with SVM
print("\nTraining SVM for Cancer Type Classification...")
svm_type = SVC(kernel='linear', random_state=42)
svm_type.fit(X_train_scaled, y_type_train)

# Predict and evaluate
y_type_pred_svm = svm_type.predict(X_test_scaled)

print("\nCancer Type Classification Results (SVM):")
print("Accuracy:", accuracy_score(y_type_test, y_type_pred_svm))
print("\nClassification Report:")
print(classification_report(y_type_test, y_type_pred_svm, target_names=le_type.classes_))

# Confusion Matrix for cancer types (SVM)
plt.figure(figsize=(10, 8))
cm_type_svm = confusion_matrix(y_type_test, y_type_pred_svm)
sns.heatmap(cm_type_svm, annot=True, fmt='d', cmap='Blues',
            xticklabels=le_type.classes_,
            yticklabels=le_type.classes_)
plt.title('Confusion Matrix - Cancer Types (SVM)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# 3. Tumor Grade Classification with SVM
print("\nTraining SVM for Tumor Grade Classification...")
svm_grade = SVC(kernel='linear', random_state=42)
svm_grade.fit(X_train_scaled, y_grade_train)

# Predict and evaluate
y_grade_pred_svm = svm_grade.predict(X_test_scaled)

print("\nTumor Grade Classification Results (SVM):")
print("Accuracy:", accuracy_score(y_grade_test, y_grade_pred_svm))
print("\nClassification Report:")
print(classification_report(y_grade_test, y_grade_pred_svm, target_names=le_grade.classes_))

# Confusion Matrix for tumor grades (SVM)
plt.figure(figsize=(10, 8))
cm_grade_svm = confusion_matrix(y_grade_test, y_grade_pred_svm)
sns.heatmap(cm_grade_svm, annot=True, fmt='d', cmap='Blues',
            xticklabels=le_grade.classes_,
            yticklabels=le_grade.classes_)
plt.title('Confusion Matrix - Tumor Grades (SVM)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# 4. Feature Importance Analysis
def get_svm_feature_importance(svm_model, feature_names, class_labels):
    importance_per_class = {}
    for i, class_label in enumerate(class_labels):
        if len(class_labels) == 2 and i == 1:
            continue
        coef = svm_model.coef_[i] if len(class_labels) > 2 else svm_model.coef_[0]
        import sys

In [ ]:
#The model employs a Random Forest Classifier to analyze breast cancer gene expression data, specifically addressing class imbalance through the use of computed class weights for both cancer types and tumor grades. By fitting the model with these weights, it enhances the model's sensitivity to underrepresented classes during training. After training, the model evaluates its performance using accuracy metrics, classification reports, and confusion matrices, visualizing the results to better understand prediction accuracy across different categories.
#Additionally, it provides insights into feature importance, identifying the most influential genes for each classification tas
#MODEL4
# weighted Machine Learning Analysis for Breast Cancer Gene Expression Data
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight

# Using the preprocessed data from previous code
# Assuming we have: X_train_scaled, X_test_scaled, y_type_train, y_type_test, y_grade_train, y_grade_test

# 1. Calculate class weights for cancer types
class_weights_type = compute_class_weight('balanced', classes=np.unique(y_type_train), y=y_type_train)
class_weight_dict_type = dict(enumerate(class_weights_type))

# Train and evaluate cancer type classifier
rf_type = RandomForestClassifier(n_estimators=100, random_state=42, class_weight=class_weight_dict_type)
rf_type.fit(X_train_scaled, y_type_train)

# Predict and evaluate
y_type_pred = rf_type.predict(X_test_scaled)

print("Cancer Type Classification Results:")
print("Accuracy:", accuracy_score(y_type_test, y_type_pred))
print("\nClassification Report:")
print(classification_report(y_type_test, y_type_pred, target_names=le_type.classes_))

# Confusion Matrix for cancer types
plt.figure(figsize=(10, 8))
cm_type = confusion_matrix(y_type_test, y_type_pred)
sns.heatmap(cm_type, annot=True, fmt='d', cmap='Blues',
            xticklabels=le_type.classes_,
            yticklabels=le_type.classes_)
plt.title('Confusion Matrix - Cancer Types')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# 2. Calculate class weights for tumor grades
class_weights_grade = compute_class_weight('balanced', classes=np.unique(y_grade_train), y=y_grade_train)
class_weight_dict_grade = dict(enumerate(class_weights_grade))

# Train and evaluate tumor grade classifier
rf_grade = RandomForestClassifier(n_estimators=100, random_state=42, class_weight=class_weight_dict_grade)
rf_grade.fit(X_train_scaled, y_grade_train)

# Predict and evaluate
y_grade_pred = rf_grade.predict(X_test_scaled)

print("\nTumor Grade Classification Results:")
print("Accuracy:", accuracy_score(y_grade_test, y_grade_pred))
print("\nClassification Report:")
print(classification_report(y_grade_test, y_grade_pred, target_names=le_grade.classes_))

# Confusion Matrix for tumor grades
plt.figure(figsize=(10, 8))
cm_grade = confusion_matrix(y_grade_test, y_grade_pred)
sns.heatmap(cm_grade, annot=True, fmt='d', cmap='Blues',
            xticklabels=le_grade.classes_,
            yticklabels=le_grade.classes_)
plt.title('Confusion Matrix - Tumor Grades')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# 3. Feature Importance Analysis
# For cancer types
feature_importance_type = pd.DataFrame({
    'gene': X.columns,
    'importance': rf_type.feature_importances_
})
feature_importance_type = feature_importance_type.sort_values('importance', ascending=False)

# For tumor grades
feature_importance_grade = pd.DataFrame({
    'gene': X.columns,
    'importance': rf_grade.feature_importances_
})
feature_importance_grade = feature_importance_grade.sort_values('importance', ascending=False)

# Print top 10 important genes for each classification
print("\nTop 10 Important Genes for Cancer Type Classification:")
for i, row in feature_importance_type.head(10).iterrows():
    gene_symbol = gene_map.get(row['gene'], row['gene'])
    print(f"{gene_symbol}: {row['importance']:.4f}")

print("\nTop 10 Important Genes for Tumor Grade Classification:")
for i, row in feature_importance_grade.head(10).iterrows():
    gene_symbol = gene_map.get(row['gene'], row['gene'])
    print(f"{gene_symbol}: {row['importance']:.4f}")


In [ ]:
#This model implements a neural network using TensorFlow's
# Keras API to classify breast cancer types and tumor grades, while addressing class
 #imbalance through computed class weights. The architecture features multiple dense layers with dropout and batch normalization for improved generalization and reduced overfitting. The model employs early stopping and learning rate reduction strategies during training, enhancing its robustness. After training, the model evaluates its performance using #accuracy metrics, classification reports, and confusion matrices, providing a comprehensive comparison against previous models.

# Add these lines to one-hot encode the target variables
from tensorflow.keras.utils import to_categorical

y_type_train_cat = to_categorical(y_type_train)
y_type_test_cat = to_categorical(y_type_test)
y_grade_train_cat = to_categorical(y_grade_train)
y_grade_test_cat = to_categorical(y_grade_test)

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Compute class weights for both type and grade
class_weights_type = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_type_train),
    y=y_type_train
)
class_weights_type_dict = dict(enumerate(class_weights_type))

class_weights_grade = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_grade_train),
    y=y_grade_train
)
class_weights_grade_dict = dict(enumerate(class_weights_grade))

def create_improved_model(input_dim, output_dim):
    model = Sequential([
        Dense(512, activation='relu', input_dim=input_dim, kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Dropout(0.4),

        Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Dropout(0.4),

        Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Dropout(0.3),

        Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Dropout(0.2),

        Dense(output_dim, activation='softmax')
    ])

    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def train_and_evaluate_model(X_train, y_train, X_test, y_test, class_weights, le_classes, model_name):
    model = create_improved_model(X_train.shape[1], y_train.shape[1])

    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    )
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=0.00001,
        verbose=1
    )

    history = model.fit(
        X_train, y_train,
        validation_split=0.2,
        epochs=200,
        batch_size=16,
        callbacks=[early_stopping, reduce_lr],
        class_weight=class_weights,
        verbose=1
    )

    # Evaluate model
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)

    print(f"\n{model_name} Classification Results:")
    print("Accuracy:", accuracy_score(y_test_classes, y_pred_classes))
    print("\nClassification Report:")
    print(classification_report(y_test_classes, y_pred_classes, target_names=le_classes))

      # Plot training history
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'{model_name} Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'{model_name} Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.tight_layout()
    plt.show()

    # Plot confusion matrix
    cm = confusion_matrix(y_test_classes, y_pred_classes)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=le_classes,
                yticklabels=le_classes)
    plt.title(f'{model_name} Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

    return model, history, y_pred_classes

# Train and evaluate cancer type model
print("Training Cancer Type Classification Model...")
nn_type, history_type, y_type_pred_classes = train_and_evaluate_model(
    X_train_scaled, y_type_train_cat,
    X_test_scaled, y_type_test_cat,
    class_weights_type_dict, le_type.classes_,
    "Cancer Type"
)

# Train and evaluate tumor grade model
print("\nTraining Tumor Grade Classification Model...")
nn_grade, history_grade, y_grade_pred_classes = train_and_evaluate_model(
    X_train_scaled, y_grade_train_cat,
    X_test_scaled, y_grade_test_cat,
    class_weights_grade_dict, le_grade.classes_,
    "Tumor Grade"
)

# Compare with previous models
def compare_all_models(y_true, y_pred_rf, y_pred_svm, y_pred_nn_old, y_pred_nn_new, model_type):
    y_true_classes = np.argmax(y_true, axis=1)
    print(f"\nComparison for {model_type}:")
    print(f"Random Forest Accuracy: {accuracy_score(y_true_classes, y_pred_rf):.4f}")
    print(f"SVM Accuracy: {accuracy_score(y_true_classes, y_pred_svm):.4f}")
    print(f"Original Neural Network Accuracy: {accuracy_score(y_true_classes, y_pred_nn_old):.4f}")
    print(f"Improved Neural Network Accuracy: {accuracy_score(y_true_classes, y_pred_nn_new):.4f}")

In [ ]:
#This function analyzes the important features identified by a trained SVM model for cancer type and tumor grade classification. It computes the importance of genes based on the SVM coefficients, sorting them to identify the top five positive and negative contributors for each class. The function then evaluates and prints the average expression levels of these genes within the respective class compared to other types, calculating fold
#changes for additional insights. Finally, it saves the top 50 important genes for each class to a CSV file for further analysis.
import pandas as pd

def analyze_important_features(X, y, svm_model, gene_map, class_labels):
    # Get feature importance
    importance_per_class = {}
    for i, class_label in enumerate(class_labels):
        if len(class_labels) == 2 and i == 1:
            continue
        coef = svm_model.coef_[i] if len(class_labels) > 2 else svm_model.coef_[0]
        importance = pd.DataFrame({
            'gene': X.columns,
            'importance': coef  # Note: not using absolute value here
        })
        importance_per_class[class_label] = importance.sort_values('importance', ascending=False)

    # Analyze top genes
    print("\nTop Important Genes Analysis:")
    for class_label, importance_df in importance_per_class.items():
        print(f"\nFor {class_label}:")

        # Get top 5 positive and negative genes
        top_positive = importance_df.head(5)
        top_negative = importance_df.tail(5).iloc[::-1]

        # Analyze expression patterns
        for importance_type, top_genes in [("Higher in "+class_label, top_positive),
                                             ("Lower in "+class_label, top_negative)]:
            print(f"\n{importance_type}:")
            for _, row in top_genes.iterrows():
                gene_id = row['gene']
                gene_symbol = gene_map.get(gene_id, gene_id)
                importance = row['importance']

                # Calculate average expression in this class vs others
                mask_this_class = (y == i)
                avg_expr_this_class = X[gene_id][mask_this_class].mean()
                avg_expr_other = X[gene_id][~mask_this_class].mean()

                print(f"{gene_symbol}: "
                      f"Avg expression in {class_label}: {avg_expr_this_class:.2f}, "
                      f"Other types: {avg_expr_other:.2f}, "
                      f"Fold change: {avg_expr_this_class/avg_expr_other:.2f}")

        # Save top 50 genes to CSV
        top_50_genes = importance_df.head(50)
        top_50_genes['gene_symbol'] = top_50_genes['gene'].map(gene_map).fillna(top_50_genes['gene'])
        top_50_genes.to_csv(f"{class_label}_top_50_genes.csv", index=False)
        print(f"\nTop 50 genes for {class_label} saved to '{class_label}_top_50_genes.csv'.")

# After training the SVM model, call this function:
print("\nAnalyzing cancer type classification features:")
analyze_important_features(X, y_type_encoded, svm_type, gene_map, le_type.classes_)

print("\nAnalyzing tumor grade classification features:")
analyze_important_features(X, y_grade_encoded, svm_grade, gene_map, le_grade.classes_)